In [1]:
import os

In [2]:
reference_fasta_file = "/mnt/projects/pasteur_bacterias/refs/Salmon1418_reference.fna"

In [3]:
def sc_iter_fasta_for_refseq(file_name):
    """ Iter over fasta file."""

    header = None
    seq = []
    with open(file_name) as fh:
        data = fh.readlines()
        for line in data:
            if line.startswith(">"):
                if seq:
                    sequence = "".join(seq).upper()
                    yield (header, sequence)
                header = line.strip()[1:]
                seq = []
                continue
            seq.append(line.upper().strip())
        if seq or header:
            sequence = "".join(seq).upper()
            yield (header, sequence)

In [4]:
for header, sequence in sc_iter_fasta_for_refseq(reference_fasta_file):
    print(header, len(sequence))
    break
    
len(sequence)

AE006468.2 Salmonella enterica subsp. enterica serovar Typhimurium str. LT2, complete genome 4857450


4857450

In [5]:
def get_right_forks(kmer, graph):
    ''' Returns
    - R as a dictionary nucleotide to tf
    '''
    forks = {}
    a = graph[kmer[1:]+'A']
    if a:
        forks["A"] = a
    c = graph[kmer[1:]+'C']
    if c:
        forks["C"] = c
    t = graph[kmer[1:]+'T']
    if t:
        forks["T"] = t
    g = graph[kmer[1:]+'G']
    if g:
        forks["G"] = g
    return forks


def print_prev_R(kmer, kmer2freq, cutoff=0):
    ''' Returns:
    - R as a dictionary nucleotide to tf
    '''
    R = {}
    a = kmer2freq['A'+kmer[:-1]]
    if a and a > cutoff:
        R["A"] = a
    c = kmer2freq['C'+kmer[:-1]]
    if c and c > cutoff:
        R["C"] = c
    t = kmer2freq['T'+kmer[:-1]]
    if t and t > cutoff:
        R["T"] = t
    g = kmer2freq['G'+kmer[:-1]]
    if g and g > cutoff:
        R["G"] = g
    return R

In [25]:
get_right_forks(sequence[:23], graph)

{'G': 1}

In [19]:
from collections import defaultdict

### 101 illumina 
### 350 PE illumina 101

def get_graph(sequence, k):
    graph = defaultdict(int)
    for i in range(len(sequence)-k+1):
        kmer = sequence[i:i+k]
        graph[kmer] += 1
    return graph

In [23]:
graph = get_graph(sequence, 23)

In [16]:
def UBER_ASSEMBLER(sequence, graph, k, verbose=False):
    ''' Stupid assembler.
    '''
    start_kmer = sequence[:k]
    queue = [start_kmer]
    contigs = []
    seen_kmers = set()
    while queue:
        kmer = queue.pop(0)
        contig = kmer
        ### handle repeats
        if kmer in seen_kmers:
            continue
        seen_kmers.add(kmer)
        ### assembly kmer until fork or end
        while True:
            R = get_right_forks(kmer, graph)
            if len(R) == 1:
                contig += list(R.keys())[0]
                kmer = kmer[1:] + list(R.keys())[0]
            elif len(R) > 1:
                for nucl in R:
                    rkmer = kmer[1:] + nucl
                    queue.append(rkmer)
                break
            elif len(R) == 0:
                break
        contigs.append(contig)
        lengths = [len(x) for x in contigs]
        if verbose:
            print(len(contigs), get_n50(lengths) )
    if verbose:
        print("DONE")
#     return contigs

In [49]:
%%time
number = 23
fork_numbers = []
contig_number = []
i = 0
while i < 20:
    print(number)
    contig_number.append(number)
    graph = get_graph(sequence, number)
    
    for kmer,fork in graph.items():
        if fork > 1:
            mid_number = contig_number[-1] + (contig_number[-2] - contig_number[-1]) // 2
            if mid_number in fork_numbers:
                break
            else:
                fork_numbers.append(number)
            break
        else:
            continue
    
    if len(contig_number) < 2:
        number = number // 2
    elif contig_number[-1] > contig_number[-2]:
        number = contig_number[-2] + (contig_number[-1] - contig_number[-2]) // 2
    else:
        number = contig_number[-1] + (contig_number[-2] - contig_number[-1]) // 2
    i+=1

print(contig_number[-1])
print(fork_numbers)

10000
5000
7500
6250
6875
6562
6718
6640
6679
6659
6669


KeyboardInterrupt: 

In [32]:
check = 1000
poop = ["profp", "d,elf,", "d,emep"]
for i in range(10):
    print(check)
    if len(poop) != check:
        print("One more time!")
        check = check // 2

1000
One more time!
500
One more time!
250
One more time!
125
One more time!
62
One more time!
31
One more time!
15
One more time!
7
One more time!
3
3
